In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from itertools import repeat


from snewpy import snowglobes
from astropy import units as u
from snewpy.neutrino import Flavor, MassHierarchy, MixingParameters
from snewpy.models.ccsn import *
from snewpy.flavor_transformation import AdiabaticMSW


SNOwGLoBES_path = "/Users/sierra/Desktop/neutrino_fall_2022/dependencies/snowglobes"
SNEWPY_models_base = "/Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models"  # directory containing SNEWPY models

# set parameters that don't vary over all models
# set distance in kpc
distance = 1.0

# set SNOwGLoBES detector to use
detector = "ar40kt"

# set desired flavor transformation
transformation = 'AdiabaticMSW_NMO'


# set desired burst and remaining-epoch time
window_tstart = 0.1 # Zha
window_tend = 1.9
window_bins = 18

window_tstart_burst = -0.1 # Zha
window_tend_burst = 0.1
window_bins_burst = 20

# Ignore this, just making and combining time-step arrays
tstart = np.linspace(window_tstart, window_tend, window_bins, endpoint=False) 
tend = tstart + (window_tend - window_tstart) / window_bins 
tmid = (tstart + tend) * 0.5 

tstart_burst = np.linspace(window_tstart_burst, window_tend_burst, window_bins_burst, endpoint=False) 
tend_burst = tstart_burst + (window_tend_burst - window_tstart_burst) / window_bins_burst 
tmid_burst = (tstart_burst + tend_burst) * 0.5 

tstart = np.append(tstart_burst, tstart)
tend = np.append(tend_burst, tend)


In [6]:
matrix = [['Bollig_2016', 's27.0c'],
         ['Sukhbold_2015', 'sukhbold-LS220-s27.0.fits'],
         ['Sukhbold_2015', 'sukhbold-SFHo-s27.0.fits'],
         ['Warren_2020', 'stir_multimessenger_a1.27_m27.0.h5'],
         ['Warren_2020', 'stir_multimessenger_a1.23_m27.0.h5']]
# change these to choose model to use
modeltype = matrix[1][0]
model = matrix[1][1]
# absolute paths to new folders - change these too
new_dir_fluence = "."
new_dir_interactions = "."



modelfile = SNEWPY_models_base + "/" + modeltype + "/" + model 
outfile = modeltype+"_"+model+"_"+transformation


    # snowglobes.generate_fluence integrates the model over the specified time window(s)
    # and generates input files for SNOwGLoBES. It returns the full file path of the output file.
print("Preparing fluences ...")
tarredfile = snowglobes.generate_fluence(modelfile, modeltype, transformation, distance, outfile, tstart * u.s, tend * u.s)

    # Next, we run SNOwGLoBES. This will loop over all the fluence files in `tarredfile`.
print("Running SNOwGLoBES ...")
sim = snowglobes.simulate(SNOwGLoBES_path, tarredfile, detector_input=detector)

    # Finally, we collate SNOwGLoBES’ results into a dictionary
print("Collating results ...")
tables = snowglobes.collate(SNOwGLoBES_path, tarredfile, skip_plots=True)
print("Done\n")


## commands to move and unpack folders
print("Copy these for interactions \n")
print(f"cd {new_dir_interactions}")
print("mv " + SNEWPY_models_base + "/" + modeltype + "/" + outfile + f"_SNOprocessed.tar.gz .")
print("tar -xf " + outfile + "_SNOprocessed.tar.gz\n")

print("Copy these for fluence \n")
print(f"cd {new_dir_fluence}")
print("mv " + SNEWPY_models_base + "/" + modeltype + "/" + outfile + f".tar.bz2 .") 
print("tar -xf " + outfile + ".tar.bz2")
print("rm parameterinfo")
print("rm " + outfile + ".tar.bz2")


/Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/sukhbold-LS220-s27.0.fits
Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO
Preparing fluences ...
Collating results ...
Done

Copy these for interactions 

cd .
mv /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO_SNOprocessed.tar.gz .
tar -xf Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO_SNOprocessed.tar.gz

Copy these for fluence 

cd .
mv /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO.tar.bz2 .
tar -xf Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO.tar.bz2
rm parameterinfo
rm Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO.tar.bz2
